In [1]:
from glob import glob
path_to_subtitles = sorted(glob("./subtitles/*.ass"))

In [2]:
path_to_subtitles[:10]

['./subtitles/001_LEG_English.ass',
 './subtitles/002_LEG_English.ass',
 './subtitles/003_LEG_English.ass',
 './subtitles/004_LEG_English.ass',
 './subtitles/005_LEG_English.ass',
 './subtitles/006_LEG_English.ass',
 './subtitles/007_LEG_English.ass',
 './subtitles/008_LEG_English.ass',
 './subtitles/009_LEG_English.ass',
 './subtitles/010_LEG_English.ass']

In [3]:
scripts=[]
episode_number=[]
for path in path_to_subtitles:
    with open(path,'r') as file:
        lines = file.readlines()
        lines = lines[27:]

        rows = [",".join(line.split(",")[9:]) for line in lines]

    rows = [line.replace("\\N"," ") for line in rows]
    script = " ".join(rows)

    episode = int(path.split('_')[0].split('/')[2].strip())
    scripts.append(script)
    episode_number.append(episode)    
    

 

In [4]:
import pandas as pd
df = pd.DataFrame.from_dict({'episode':episode_number,'script':scripts})


In [5]:
df.head()

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


In [6]:
import spacy

In [7]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [9]:
doc = nlp("Sasuke went to konoha")

In [10]:
for ent in doc.ents:
    print(ent.text,ent.label_)

Sasuke PERSON
konoha GPE


In [11]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /home/suraj/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
def get_ners(script):
    script_sentences = sent_tokenize(script)

    ner_output = []

    for sentence in script_sentences:
        doc = nlp(sentence)
        ners = set()    
        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                full_name = ent.text
                first_name = full_name.split(' ')[0]
                ners.add(first_name)
        ner_output.append(list(ners))
    return ner_output                    


In [13]:
df['ners'] = df['script'].apply(get_ners)

In [14]:
window = 10
entity_relationship = []

for row in df['ners']:
    previous_entities_in_window = []

    for sentence in row:
        previous_entities_in_window.append(sentence)
        previous_entities_in_window = previous_entities_in_window[-10:]

        previous_entities_flattend = sum(previous_entities_in_window,[])

        for entity in sentence:
            for entity_in_window in previous_entities_flattend:
                if entity!= entity_in_window:
                    entity_rel = sorted([entity,entity_in_window])
                    entity_relationship.append(entity_rel)
